# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [48]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 0.9574738771891631
 1.6749724407466124
 1.3028683836842754

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [49]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.116238  1.0       0.0
 0.95322   0.114915  1.0
U factor:
3×3 Matrix{Float64}:
 0.513322  0.304063   0.857587
 0.0       0.400936   0.361843
 0.0       0.0       -0.381403

In [50]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [51]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [52]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.116238  1.0       0.0
 0.95322   0.114915  1.0

In [53]:
Alu.U

3×3 Matrix{Float64}:
 0.513322  0.304063   0.857587
 0.0       0.400936   0.361843
 0.0       0.0       -0.381403

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [54]:
A\b

3-element Vector{Float64}:
 1.0000000000000002
 1.0
 1.0

In [55]:
Alu\b

3-element Vector{Float64}:
 1.0000000000000002
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [56]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [57]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 -0.711676  -0.486849  -0.985664
  0.0        0.398257   0.358294
  0.0        0.0       -0.27695

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [58]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

In [59]:
Aqr.R

3×3 Matrix{Float64}:
 -0.711676  -0.486849  -0.985664
  0.0        0.398257   0.358294
  0.0        0.0       -0.27695

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [60]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.6178753889484818
 -0.3873723236829061
  2.6880038051345467
vectors:
3×3 Matrix{Float64}:
  0.813484   -0.351969  -0.462993
 -0.579238   -0.561815  -0.590633
 -0.0522317   0.748653  -0.660901

The values and the vectors can be extracted from the Eigen type by special indexing

In [61]:
AsymEig.values

3-element Vector{Float64}:
 -0.6178753889484818
 -0.3873723236829061
  2.6880038051345467

In [62]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.813484   -0.351969  -0.462993
 -0.579238   -0.561815  -0.590633
 -0.0522317   0.748653  -0.660901

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [63]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           6.66134e-16  8.88178e-16
  8.88178e-16   1.0          1.11022e-15
 -4.44089e-16  -1.11022e-15  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [64]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [65]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [66]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.8802653698935093

In [67]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [68]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [69]:
@time eigvals(Asym);

  0.059832 seconds (11 allocations: 7.989 MiB)


In [70]:
@time eigvals(Asym_noisy);

  0.420537 seconds (14 allocations: 7.928 MiB, 1.75% gc time)


In [71]:
@time eigvals(Asym_explicit);

  0.112748 seconds (11 allocations: 7.989 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [72]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.548130 seconds (17 allocations: 183.106 MiB, 6.61% gc time)


6.456234583545239

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [73]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [74]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
  1    1//5  7//10
 1//5  3//5  7//10
 1//2  3//5  3//5

In [75]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 19//10
  3//2
 17//10

In [76]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1
 1
 1

In [77]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
  1      0     0
 1//5    1     0
 1//2  25//28  1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1   1//5    7//10
 0  14//25  14//25
 0    0     -1//4

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [78]:
using LinearAlgebra

In [79]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
A_eigv = eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514488
  542.467730146614
vectors:
5×5 Matrix{Float64}:
 -0.147575   0.647178   0.010882    0.548903   -0.507907
 -0.256795  -0.173068   0.834628   -0.239864   -0.387253
 -0.185537   0.239762  -0.422161   -0.731925   -0.440631
  0.819704  -0.247506  -0.0273194   0.0366447  -0.514526
 -0.453805  -0.657619  -0.352577    0.322668   -0.364928

In [80]:
A_eigv = eigen(Symmetric(A))
# strange why numerically thee is a difference with their result. But minor difference.

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514488
  542.467730146614
vectors:
5×5 Matrix{Float64}:
 -0.147575   0.647178   0.010882    0.548903   -0.507907
 -0.256795  -0.173068   0.834628   -0.239864   -0.387253
 -0.185537   0.239762  -0.422161   -0.731925   -0.440631
  0.819704  -0.247506  -0.0273194   0.0366447  -0.514526
 -0.453805  -0.657619  -0.352577    0.322668   -0.364928

In [81]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [93]:
A_eigvalues = A_eigv.values
A_diag = zeros(length(A_eigvalues),length(A_eigvalues))

A_diag += Diagonal(A_eigv.values)

5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [94]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

LoadError: AssertionError: A_diag == [-128.493 0.0 0.0 0.0 0.0; 0.0 -55.8878 0.0 0.0 0.0; 0.0 0.0 42.7522 0.0 0.0; 0.0 0.0 0.0 87.1611 0.0; 0.0 0.0 0.0 0.0 542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [98]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
# They don't request the decompositioning, but just to take the lower triangular matrix
# Alu = lu(A)
# A_lowertri = Alu.L

A_lowertri = LowerTriangular(A)


5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [99]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]